### Importing the Libraries

In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

### Loading the Dataset, applying Padding and Splitting the data

In [ ]:
# load the dataset but only keep the top n words, zero the rest
top_words = 8000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

# pad dataset to a maximum review length in words
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

X = numpy.concatenate((X_train, X_test), axis=0)
y = numpy.concatenate((y_train, y_test), axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # split into train and test sets.

### Defining the Model Architecture

In [ ]:
model = Sequential()
model.add(Embedding(top_words, 300, input_length=max_words))
model.add(Conv1D(filters=256, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(.8))
model.add(Dense(250, activation='relu'))
model.add(Dropout(.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_10 (Conv1D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_10 (MaxPooling1D)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_11 (MaxPooling1D)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


### Training the Model

In [ ]:
history=model.fit(X_train, y_train, validation_split=0.2, epochs=8, batch_size=128, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/8
219/219 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.5294 - loss: 0.6768 - val_accuracy: 0.8754 - val_loss: 0.2942
Epoch 2/8
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 48ms/step - accuracy: 0.8872 - loss: 0.2834 - val_accuracy: 0.8986 - val_loss: 0.2492
Epoch 3/8
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - accuracy: 0.9217 - loss: 0.1979 - val_accuracy: 0.8987 - val_loss: 0.2515
Epoch 4/8
219/219 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - accuracy: 0.9465 - loss: 0.1425 - val_accuracy: 0.9009 - val_loss: 0.2691
Epoch 5/8
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.9636 - loss: 0.1019 - val_accuracy: 0.8979 - val_loss: 0.3247
Epoch 6/8
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.9759 - loss: 0.0721 - val_accuracy: 0.8909 - val_loss: 0.3556
Epoch 7/8
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.9800 - loss: 0.0584 - val_accuracy: 0.8919 - val_loss: 0.4482
Epoch 8/8
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.9830 - loss: 0.0489 - val_accu

### Evaluating the Model

In [ ]:
test_size=len(X_test)
y_test_pred = model.predict(X_test)
for i in range(test_size):
    if y_test_pred[i]>0.4:
        y_test_pred[i]=1
    else:
        y_test_pred[i]=0

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_test_pred)
print(cm)

[[6341 1057]
 [ 503 7099]]


In [ ]:
tp=cm[0][0]
fn=cm[0][1]
fp=cm[1][0]
tn=cm[1][1]

errorrate=((fp+fn)/(tp+fn+fp+tn))
accuracy=((tp+tn)/(tp+fn+fp+tn))
recall=(tp/(tp+fn))
specificity=(tn/(tn+fp))
precision=(tp/(tp+fp))
# fpr=1-specificity
f1_score=(2*precision*recall)/(precision+recall)

In [ ]:
print("Error Rate")
print(errorrate * 100)
print("Accuracy")
print(accuracy * 100)
print("Recall")
print(recall * 100)
print("Specificity")
print(specificity * 100)
print("Precision")
print(precision * 100)
'''
print("fpr")
print(fpr)
'''
print("F1 Score")
print(f1_score * 100)

Error Rate
10.4
Accuracy
89.60000000000001
Recall
85.71235469045688
Specificity
93.38332017890029
Precision
92.65049678550555
F1 Score
89.04648223564107


### Saving the Model

In [ ]:
model.save("cnnmodel.h5")